
## <p style="text-align: center;">MIS 284N - Big Data and Distributed Programming</p>
## <p style="text-align: center;">Project 3 - Machine Learning using Tensorflow and AWS</p>
## <p style="text-align: center;">Total points: 100</p>
## <p style="text-align: center;">Due: Thursday, October 19 submitted via Canvas by 11:59 pm</p>

Your project should be written in a **Jupyter notebook**. You may work in groups of two if you wish. Only one student per team needs to submit the assignment on Canvas.  But be sure to include name and UTEID for both students.

Also, please make sure your code runs and the graphics (and anything else) are displayed in your notebook before submitting. (%matplotlib inline)

This project is on Tensorflow, its usage, Cloud services and why they are helpful in analyzing Big Data.

In this Project, we are trying to analyze crime data. This data is taken from <a href=https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2/data> here </a>. You have to download data from here, it should be around 1.6 GB in size. 

You have to predict type of crime that can happen given the features.

You have to complete all the four tasks and report your observations for each if any. There is a bonus task as well, it has two options and you can choose any one of it.

# Task 1 - Read the data and filter the data you need

This is open ended, you can use any features you want and drop any one of them. The 'Primary Type' is our target variable which we will be predicting for.
Split the data into training and testing in the ratio of 80:20.

In [2]:
import tensorflow as tf
import numpy as np

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import pandas as pd

In [3]:
data=pd.read_csv("Crimes-2001-present.csv")

In [4]:
data.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10000092,HY189866,03/18/2015 07:44:00 PM,047XX W OHIO ST,041A,BATTERY,AGGRAVATED: HANDGUN,STREET,False,False,...,28.0,25.0,04B,1144606.0,1903566.0,2015,02/10/2018 03:50:01 PM,41.891399,-87.744385,"(41.891398861, -87.744384567)"
1,10000094,HY190059,03/18/2015 11:00:00 PM,066XX S MARSHFIELD AVE,4625,OTHER OFFENSE,PAROLE VIOLATION,STREET,True,False,...,15.0,67.0,26,1166468.0,1860715.0,2015,02/10/2018 03:50:01 PM,41.773372,-87.665319,"(41.773371528, -87.665319468)"
2,10000095,HY190052,03/18/2015 10:45:00 PM,044XX S LAKE PARK AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,...,4.0,39.0,08B,1185075.0,1875622.0,2015,02/10/2018 03:50:01 PM,41.813861,-87.596643,"(41.81386068, -87.596642837)"
3,10000096,HY190054,03/18/2015 10:30:00 PM,051XX S MICHIGAN AVE,0460,BATTERY,SIMPLE,APARTMENT,False,False,...,3.0,40.0,08B,1178033.0,1870804.0,2015,02/10/2018 03:50:01 PM,41.800802,-87.622619,"(41.800802415, -87.622619343)"
4,10000097,HY189976,03/18/2015 09:00:00 PM,047XX W ADAMS ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,False,False,...,28.0,25.0,03,1144920.0,1898709.0,2015,02/10/2018 03:50:01 PM,41.878065,-87.743354,"(41.878064761, -87.743354013)"


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6720963 entries, 0 to 6720962
Data columns (total 22 columns):
ID                      int64
Case Number             object
Date                    object
Block                   object
IUCR                    object
Primary Type            object
Description             object
Location Description    object
Arrest                  bool
Domestic                bool
Beat                    int64
District                float64
Ward                    float64
Community Area          float64
FBI Code                object
X Coordinate            float64
Y Coordinate            float64
Year                    int64
Updated On              object
Latitude                float64
Longitude               float64
Location                object
dtypes: bool(2), float64(7), int64(3), object(10)
memory usage: 1.0+ GB


In [6]:
data.isnull().sum()


ID                           0
Case Number                  4
Date                         0
Block                        0
IUCR                         0
Primary Type                 0
Description                  0
Location Description      3944
Arrest                       0
Domestic                     0
Beat                         0
District                    47
Ward                    614853
Community Area          616028
FBI Code                     0
X Coordinate             60187
Y Coordinate             60187
Year                         0
Updated On                   0
Latitude                 60187
Longitude                60187
Location                 60187
dtype: int64

In [7]:
data.dropna(axis=0, how='any', inplace=True)

In [9]:
data['date_parsed'] = pd.to_datetime(data['Date'])

In [10]:
data["dayofweek"]=data['date_parsed'].dt.weekday

In [11]:
data["hour"]=data['date_parsed'].dt.hour

In [12]:
data["Primary Type"].unique()

array(['BATTERY', 'OTHER OFFENSE', 'ROBBERY', 'NARCOTICS',
       'CRIMINAL DAMAGE', 'WEAPONS VIOLATION', 'THEFT', 'BURGLARY',
       'MOTOR VEHICLE THEFT', 'PUBLIC PEACE VIOLATION', 'ASSAULT',
       'CRIMINAL TRESPASS', 'CRIM SEXUAL ASSAULT',
       'INTERFERENCE WITH PUBLIC OFFICER', 'ARSON', 'DECEPTIVE PRACTICE',
       'LIQUOR LAW VIOLATION', 'KIDNAPPING', 'SEX OFFENSE',
       'OFFENSE INVOLVING CHILDREN', 'PROSTITUTION', 'HOMICIDE',
       'GAMBLING', 'INTIMIDATION', 'STALKING', 'OBSCENITY',
       'PUBLIC INDECENCY', 'HUMAN TRAFFICKING',
       'CONCEALED CARRY LICENSE VIOLATION', 'OTHER NARCOTIC VIOLATION',
       'NON - CRIMINAL', 'NON-CRIMINAL',
       'NON-CRIMINAL (SUBJECT SPECIFIED)', 'RITUALISM'], dtype=object)

In [13]:
data["Primary Type"]=data["Primary Type"].replace({'NON - CRIMINAL': 'NON-CRIMINAL'})
data["Primary Type"]=data["Primary Type"].replace({'NON-CRIMINAL (SUBJECT SPECIFIED)': 'NON-CRIMINAL'})

In [14]:
label=data["Primary Type"]

In [15]:
data["District"].unique()

array([11.,  7.,  2.,  9.,  5.,  6.,  4., 12., 15., 14., 20., 10., 22.,
        8., 17.,  3., 18., 16., 25.,  1., 24., 19., 31., 21.])

In [17]:
data["Location Description"].unique()

array(['STREET', 'APARTMENT', 'SIDEWALK', 'RESIDENCE PORCH/HALLWAY',
       'PARKING LOT/GARAGE(NON.RESID.)', 'CTA GARAGE / OTHER PROPERTY',
       'DRIVEWAY - RESIDENTIAL', 'GAS STATION', 'OTHER',
       'GROCERY FOOD STORE', 'ALLEY', 'BAR OR TAVERN',
       'VEHICLE NON-COMMERCIAL', 'RESIDENCE',
       'FACTORY/MANUFACTURING BUILDING', 'CTA BUS STOP',
       'SCHOOL, PUBLIC, BUILDING', 'DRUG STORE',
       'COMMERCIAL / BUSINESS OFFICE', 'RESIDENCE-GARAGE',
       'SMALL RETAIL STORE', 'NURSING HOME/RETIREMENT HOME',
       'VACANT LOT/LAND', 'RESTAURANT', 'HOSPITAL BUILDING/GROUNDS',
       'DEPARTMENT STORE', 'CHURCH/SYNAGOGUE/PLACE OF WORSHIP',
       'SCHOOL, PUBLIC, GROUNDS', 'HOTEL/MOTEL', 'MEDICAL/DENTAL OFFICE',
       'SCHOOL, PRIVATE, BUILDING', 'CTA BUS',
       'RESIDENTIAL YARD (FRONT/BACK)', 'GOVERNMENT BUILDING/PROPERTY',
       'DAY CARE CENTER', 'AIRPORT/AIRCRAFT', 'CAR WASH', 'BANK',
       'TAVERN/LIQUOR STORE', 'COLLEGE/UNIVERSITY RESIDENCE HALL',
       'COLLEGE/

In [16]:

a=[]
b=[]
c=[]
d=[]
e=[]
f=[]
for i in data["Location Description"].unique():
    if "AIR" in i:
        a.append(i)
    elif "SCHOOL" in i or "UNIVERSITY" in i:
        b.append(i)
    elif "BUS" in i or "AUTO" in i or"TAXI" in i or"TRAIN"in i or"UBER" in i or"CTA"in i or"TRANSPORTATION" in i or"VEHICLE" in i or "MOTEL"in i or"TRUCK"in i:
        c.append(i)
    elif "STREET" in i or "SIDEWALK" in i or "WAY" in i or "GROUNDS" in i or "ALLEY" in i or "PARKING" in i or "FOREST" in i or "LAKE" in i or "RIVER" in i or "YARD" in i or "BRIDGE" in i or "FARM" in i or "SEWER" in i or "LAND" in i or "GARAGE" in i or "LAGOON" in i or "CEMETARY" in i:
        d.append(i)
    elif i=="OTHER" or i=="DUMPSTER" or i=='HORSE STABLE':
        e.append(i)
    else:
        f.append(i)

        

In [17]:
data["Location Description"].replace(a,'AIRPORT', inplace= True)

In [18]:
data["Location Description"].replace(b,'SCHOOL', inplace= True)
data["Location Description"].replace(c,'TRANSPORTATION', inplace= True)
data["Location Description"].replace(d,'OUTDOOR', inplace= True)
data["Location Description"].replace(e,'OTHER', inplace= True)
data["Location Description"].replace(f,'INDOOR', inplace= True)

In [37]:
data["Location Description"].unique()

array(['OUTDOOR', 'INDOOR', 'TRANSPORTATION', 'OTHER', 'SCHOOL',
       'AIRPORT'], dtype=object)

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6051090 entries, 0 to 6720962
Data columns (total 25 columns):
ID                      int64
Case Number             object
Date                    object
Block                   object
IUCR                    object
Primary Type            object
Description             object
Location Description    object
Arrest                  bool
Domestic                bool
Beat                    int64
District                float64
Ward                    float64
Community Area          float64
FBI Code                object
X Coordinate            float64
Y Coordinate            float64
Year                    int64
Updated On              object
Latitude                float64
Longitude               float64
Location                object
date_parsed             datetime64[ns]
dayofweek               int64
hour                    int64
dtypes: bool(2), datetime64[ns](1), float64(7), int64(5), object(10)
memory usage: 1.1+ GB


In [24]:
data=data.set_index("date_parsed")

In [8]:
features=["Location Description","Arrest","Domestic","District","dayofweek","hour"]


In [31]:
data.to_csv("data_clean")

# Task 2 - Convert the features in a form that can be given as input to tensorflow library/functions

Here you have to search a little bit. The data that we are using has data other than numerical values. Tensors take two kinds of values, either numeric or categorical. So, you have to convert the given data into a logical form which can be given to tensors.

In [4]:
data=pd.read_csv('data_clean')

In [5]:
len(data["Primary Type"].unique())

32

In [6]:
label=pd.get_dummies(data["Primary Type"])

In [9]:
X=data[features]

In [10]:
X=pd.get_dummies(X,columns=features)

In [23]:
data

,date_parsed,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,...,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,dayofweek,hour
0,2015-03-18 19:44:00,10000092,HY189866,03/18/2015 07:44:00 PM,047XX W OHIO ST,041A,BATTERY,AGGRAVATED: HANDGUN,OUTDOOR,False,...,04B,1144606.0,1903566.0,2015,02/10/2018 03:50:01 PM,41.891399,-87.744385,"(41.891398861, -87.744384567)",2,19
1,2015-03-18 23:00:00,10000094,HY190059,03/18/2015 11:00:00 PM,066XX S MARSHFIELD AVE,4625,OTHER OFFENSE,PAROLE VIOLATION,OUTDOOR,True,...,26,1166468.0,1860715.0,2015,02/10/2018 03:50:01 PM,41.773372,-87.665319,"(41.773371528, -87.665319468)",2,23
2,2015-03-18 22:45:00,10000095,HY190052,03/18/2015 10:45:00 PM,044XX S LAKE PARK AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,INDOOR,False,...,08B,1185075.0,1875622.0,2015,02/10/2018 03:50:01 PM,41.813861,-87.596643,"(41.81386068, -87.596642837)",2,22
3,2015-03-18 22:30:00,10000096,HY190054,03/18/2015 10:30:00 PM,051XX S MICHIGAN AVE,0460,BATTERY,SIMPLE,INDOOR,False,...,08B,1178033.0,1870804.0,2015,02/10/2018 03:50:01 PM,41.800802,-87.622619,"(41.800802415, -87.622619343)",2,22
4,2015-03-18 21:00:00,10000097,HY189976,03/18/2015 09:00:00 PM,047XX W ADAMS ST,031A,ROBBERY,ARMED: HANDGUN,OUTDOOR,False,...,03,1144920.0,1898709.0,2015,02/10/2018 03:50:01 PM,41.878065,-87.743354,"(41.878064761, -87.743354013)",2,21
5,2015-03-18 22:00:00,10000098,HY190032,03/18/2015 10:00:00 PM,049XX S DREXEL BLVD,0460,BATTERY,SIMPLE,INDOOR,False,...,08B,1183018.0,1872537.0,2015,02/10/2018 03:50:01 PM,41.805443,-87.604284,"(41.805443345, -87.604283976)",2,22
6,2015-03-18 23:00:00,10000099,HY190047,03/18/2015 11:00:00 PM,070XX S MORGAN ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,INDOOR,False,...,08B,1170859.0,1858210.0,2015,02/10/2018 03:50:01 PM,41.766403,-87.649296,"(41.766402779, -87.649296123)",2,23
7,2015-03-18 21:35:00,10000100,HY189988,03/18/2015 09:35:00 PM,042XX S PRAIRIE AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,INDOOR,False,...,08B,1178746.0,1876914.0,2015,02/10/2018 03:50:01 PM,41.817553,-87.619819,"(41.817552577, -87.619818523)",2,21
8,2015-03-18 22:09:00,10000101,HY190020,03/18/2015 10:09:00 PM,036XX S WOLCOTT AVE,1811,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,OUTDOOR,True,...,18,1164279.0,1880656.0,2015,02/10/2018 03:50:01 PM,41.828138,-87.672782,"(41.828138428, -87.672782106)",2,22
9,2015-03-18 21:25:00,10000104,HY189964,03/18/2015 09:25:00 PM,097XX S PRAIRIE AVE,0460,BATTERY,SIMPLE,OUTDOOR,False,...,08B,1179637.0,1840444.0,2015,02/10/2018 03:50:01 PM,41.717455,-87.617663,"(41.71745472, -87.617663257)",2,21


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X, label, test_size=0.2, random_state=0)

# Task 3 - Build the model

Once the features and tensors are made, you have to build a machine learning model which, given the features predicts the type of crime. 
Report the accuracy of the model.

In [13]:
y_train=tf.cast(y_train, tf.float32)

In [14]:
learning_rate = 0.01
training_epochs = 10
batch_size = 100000
display_step = 1

n_samples = X_train.shape[0]
n_features = 238
n_class = 32
x = tf.placeholder(tf.float32, [None, n_features])
y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.zeros([n_features, n_class]))
b = tf.Variable(tf.zeros([n_class]))

pred = tf.matmul(x, W) + b

cost = tf.nn.sigmoid_cross_entropy_with_logits(logits=pred, labels=y)


In [17]:

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
init = tf.initialize_all_variables()


In [19]:
import timeit

In [20]:
%%time
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(n_samples / batch_size)
        for i in range(total_batch):
            _, c = sess.run([optimizer, cost], 
                            feed_dict={x: X_train[i * batch_size : (i+1) * batch_size], 
                                      y: y_train[i * batch_size : (i+1) * batch_size, :].eval()})
            avg_cost = c / total_batch

        if (epoch+1) % display_step == 0:
            print("Epoch:", "%04d" % (epoch+1), "cost=", avg_cost)

    print("Optimization Finished!")

    print("Testing Accuracy:", accuracy.eval({x: X_train, y:y_train.eval()}))


KeyboardInterrupt: 

# Task 4 - Run the model on your laptop and AWS cloud and observe the time taken for reading the data, training and testing. 

Report your observations. Due to more RAM needed for the same, it might be the case that it doesn't run on your laptop, so in that case report your laptop configuration and state it didnt run.

# Bonus Tasks

# Option 1: Try to build a Neural Network model, train on the same features and report the accuracy.
Report your observations.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline

In [ ]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)

In [ ]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [ ]:
# define a Deeper model
def larger_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
    model.add(Dense(6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
numpy.random.seed(seed)
estimators = []
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

# Option 2: Try to predict other things like Arrest or Description of Crime. 

You can change the features in this task. Report your observations.